Consulta o banco de dados e salva em csv cada tabela

In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd
from datetime import timedelta
import os

save_path = r"endereço de salvamento"
# Database connection
engine = create_engine(
    "endereço base de dados", 
    echo=False
)
#lista de nomes possíveis para as colunas
nomes_coluna = ["Impp", "IV Point[3f612]-i", "IV Point[a9112]-i"]

# Defina o limite de tempo
tempo_limite = pd.Timestamp.now().floor('s') - timedelta(minutes=33)
print(tempo_limite)

# Query to get table names
table_query = text(
    "SELECT tablename FROM pg_tables WHERE tablename LIKE '%_POS%' AND tablename NOT LIKE '%OLD%' AND schemaname = 'public'"
)

with engine.connect() as conn:
    # Execute the query to get table names
    table_names = conn.execute(table_query).fetchall()

    # Iterate through table names and export each to a CSV file
    for table_name in table_names:
        print(table_name)
        table_name = table_name[0]  # Get the table name string from the result

        # Create a flag to check if any column was found
        column_found = False

        # Iterate through possible column names
        for col_name in nomes_coluna:
            # Check if the column exists in the table
            query = text(f'SELECT column_name FROM information_schema.columns WHERE table_name = :table_name AND column_name = :col_name')
            result = conn.execute(query, {'table_name': table_name, 'col_name': col_name}).fetchone()
            
            if result:  # If the column exists
                query = text(f'SELECT "TIMESTAMP", "{col_name}" FROM "{table_name}"  WHERE "TIMESTAMP" >= :tempo_limite')
                df = pd.read_sql(query, conn, params={'tempo_limite': tempo_limite})  # Read the table into a DataFrame
                csv_file_name = f"\\{table_name.replace('/', '_')}-{col_name}.csv"
                final_save_path = save_path + csv_file_name        

                df.to_csv(final_save_path, index=False)  # Save DataFrame to CSV
                column_found = True  # Set flag to True

        # Check if no columns were found
        if not column_found:
            print(f"No matching columns found in table {table_name}")

print("CSV files have been created.")


Itera sobre cada arquivo e cria uma nova coluna para indicar se o valor é o mesmo da linha anterior

In [ ]:
pasta = r"local dos arquivos"

# Lista todos os arquivos na pasta
arquivos = [arquivo for arquivo in os.listdir(pasta) if arquivo.endswith('.csv')]

def verificar_consecutivos_iguais(df, nome_arquivo):
    # Calcula a diferença entre valores consecutivos na coluna 'Corrente' e verifica se é igual a 0
    df["Flag"] = df['Corrente'].diff() == 0
    # Cria uma condição onde o valor atual na coluna 'Flag' é True E o valor anterior na coluna 'Flag' é True
    filterFreeze = df["Flag"] & df["Flag"].shift(1).fillna(False)

    # Verifica se existem 2 ou 3 linhas consecutivas com valor 'True' na coluna "Flag"
    if ((filterFreeze.rolling(window=2).sum() >= 2) | (filterFreeze.rolling(window=3).sum() >= 3)).any():
        resposta_travado = f"Existem no mínimo 3 pontos consecutivos com mesmo valor. No arquivo {nome_arquivo}"
        dado_travado = True
        resposta_nao_travado = None
    else:
        resposta_nao_travado = f"Não existem no mínimo 3 pontos consecutivos com mesmo valor. No arquivo {nome_arquivo}"
        dado_travado = False
        resposta_travado = None
    return resposta_travado, resposta_nao_travado, dado_travado

# Itera sobre cada nome de arquivo na lista 'arquivos'
for arquivo in arquivos:
    # Lê o arquivo CSV
    df = pd.read_csv(os.path.join(pasta, arquivo))
    # Separa o nome do arquivo e a extensão
    nome_arquivo, extensao = os.path.splitext(arquivo)

    # Renomeia as colunas do DataFrame
    df.rename(columns={"Impp": "Corrente", "IV Point[3f612]-i": "Corrente", "IV Point[a9112]-i": "Corrente"}, inplace=True)
    
    # Verifica se existem 2 ou 3 linhas consecutivas com valor 'True' na coluna "Flag"
    resposta_travado, resposta_nao_travado, dado_travado = verificar_consecutivos_iguais(df, nome_arquivo)
    
    if dado_travado:
        print("Dado travado:", resposta_travado)
    else:
        print("Dado não travado:", resposta_nao_travado)
